In [1]:
import time

import duckdb
import pandas as pd
import plotly.express as px
from plotly_calplot import calplot

from common.env_variables import DUCKDB_DWH_FILE
def display_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, "expand_frame_repr", False, "display.float_format", '${:,.2f}'.format):
        display(df.fillna('.'))

conn = duckdb.connect(DUCKDB_DWH_FILE, read_only=True)

In [2]:
df = conn.execute(f'''
SELECT
    company_name,
    COUNT('job_id') as job_count
FROM curated.job_description
GROUP BY 1
ORDER BY 2 DESC
LIMIT 100
''').df()
display_df(df)

,company_name,job_count
0,Deutsche Bahn AG,15727
1,Bosch Gruppe,10189
2,Bertrandt AG,6039
3,HAPEKO Hanseatisches Personalkontor GmbH,5989
4,ZEISS,5638
5,Mercedes - Benz AG,5214
6,meinestadt.de,5063
7,StepStone GmbH,4982
8,FERCHAU GmbH,4826
9,IU Internationale Hochschule,3909


In [3]:
start = time.time()
df = conn.execute(f'''
SELECT
    online_at,
    COUNT('a.job_id') as job_count
FROM curated.job_description a
INNER JOIN curated.job_online b ON (a.job_id = b.job_id)
WHERE
    a.company_name = 'Bosch Gruppe'
GROUP BY 1
ORDER BY 1 DESC
LIMIT 5
''').df()
print(time.time() - start)
display_df(df)

0.8286669254302979


,online_at,job_count
0,2022-09-24,2247
1,2022-09-23,2256
2,2022-09-22,2254
3,2022-09-21,2229
4,2022-09-20,2252


In [4]:
start = time.time()
df = conn.execute(f'''
WITH bosch_jobs AS (
    SELECT
        job_id
    FROM curated.job_description a
    WHERE
        a.company_name = 'Bosch Gruppe'
)
SELECT
     online_at
    ,COUNT('job_id') AS job_count
FROM curated.job_online
WHERE job_id in (
    SELECT job_id
    FROM bosch_jobs
)
GROUP BY 1
ORDER BY 1 DESC
''').df()
print(time.time() - start)
display_df(df)

0.6998307704925537


,online_at,job_count
0,2022-09-24,1943
1,2022-09-23,1954
2,2022-09-22,1951
3,2022-09-21,1929
4,2022-09-20,1943
5,2022-09-19,1921
6,2022-09-18,1944
7,2022-09-17,1996
8,2022-09-16,1999
9,2022-09-15,1968


In [5]:
start = time.time()
df = conn.execute(f'''
  WITH all_company AS (
SELECT DISTINCT company_name
  FROM curated.job_description
),
       car_company AS (
SELECT company_name
  FROM all_company
 WHERE company_name ilike 'BMW%' OR
       company_name ilike 'Audi%' OR
       company_name ilike 'Volkswagen%' OR
       company_name ilike 'Mercedes%'
),
       car_job_description AS (
SELECT a.company_name,
       a.job_id
  FROM curated.job_description a
  JOIN car_company b
    ON (a.company_name = b.company_name)
)
SELECT company_name,
       COUNT('job_id') AS job_count
  FROM car_job_description
 GROUP BY 1
 ORDER BY 2 DESC
LIMIT 100
''').df()
print(time.time() - start)
display_df(df)

1100.8648447990417


,company_name,job_count
0,Mercedes - Benz AG,5214
1,BMW Group,2186
2,Volkswagen Financial Services AG,663
3,Mercedes-AMG GmbH,508
4,Mercedes-Benz Tech Innovation,500
5,Mercedes-Benz Mobility AG,368
6,Mercedes-Benz Group AG,347
7,Mercedes-Benz Consulting GmbH,254
8,Mercedes-Benz Bank AG,227
9,Volkswagen Infotainment GmbH,211


In [6]:
conn.close()